In [ ]:
import matplotlib.pyplot as plt
import pesummary
from pesummary.io import read
from pesummary.io import write
print(pesummary.__version__)
import h5py
import glob
import os
import shutil
import json
import bilby
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
## Converts .h5 files in directory to JSON files.

for file in glob.glob('./*.h5'):
    event = file.split('.')[1][1:]
    if './'+event+'.json' not in glob.glob('./*.json'):
        print(event)
        conversion = False
        with open('skeleton.json', 'r') as f:
            skeleton = json.load(f)
            data = read(file)
            waveforms = ['C01:IMRPhenomPXHM', 'C01:SEOBNRv4PHM', 'C01:SEOBNRv4P', 'C01:IMRPhenomPv2']
            for waveform in waveforms:
                if waveform in list(data.samples_dict.keys()):
                    conversion = True
                    print('Using samples from', waveform)
                    parameters = list(skeleton['posterior']['content']) + list(data.samples_dict[waveform].keys())
                    for parameter in parameters:
                        try:
                            skeleton['posterior']['content'][parameter] = list(data.samples_dict[waveform][parameter])
                        except KeyError:
                            skeleton['posterior']['content'].pop(parameter)
                    break
                else:
                    print(waveform, 'waveform not present')
            if conversion:
                skeleton['label'] = event + '_merge'

                extrinsic_params = ['outdir', 'sampler', 'log_evidence', 'log_evidence_err', 'log_noise_evidence',
                                    'log_bayes_factor']
                for param in extrinsic_params:
                    skeleton[param] = 'Data does not exist'

                with open(event + '.json', 'w') as g:
                    json.dump(skeleton, g)
                    print(event+'.json', 'file created.')